In [1]:
#import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#read in the data
data = pd.read_csv('energydata_complete.csv')
#view the first 5 rows of the dataset
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [6]:
#remove date and lights columns
new_data = data.drop(['date','lights'], axis = 1).head()

In [15]:
#Normalize the dataset using the min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
norm_data = pd.DataFrame(scaler.fit_transform(new_data), columns=new_data.columns)

In [29]:
#define the independent and predictor variables
x = norm_data[["T2"]]
y = norm_data[["T6"]]
#split the dataset into training and test sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [31]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
#fit the model to the training dataset
linear_model.fit(x_train, y_train)
#obtain predictions
predictions = linear_model.predict(x_test)

In [32]:
#mean absolute error to 2 decimal places
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predictions)
round(mae, 2)

0.35

In [33]:
#Residual Sum of Squares to 2 decimal places
rss = np.sum(np.square(y_test - predictions))
round(rss, 2)

T6    0.27
dtype: float64

In [34]:
#Root Mean Square Error to 3 decimal places
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predictions))
round(rmse, 3)

0.371

In [35]:
#Coeff of determination or R squared in 2 decimal places
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predictions)
round(r2_score, 2) 

-0.1

In [36]:
def get_weights_df(model, feat, col_name):
#this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [44]:
linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight')

Exception: Data must be 1-dimensional

In [39]:
#train a Ridge Regression with alpha 0.4
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)

Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [40]:
#Is there a change in the Root Mean Square Error on test set
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predictions))
round(rmse, 3)

0.371

In [41]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [42]:
lasso_weights_df = get_weights_df(lasso_reg, x_train, 'Lasso_weight')

In [43]:
lasso_weights_df

,Features,Lasso_weight
0,T2,0.0


In [45]:
#Is there a change in the Root Mean Square Error on test set
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predictions))
round(rmse, 3)

0.371